In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import LSTM
#from keras.wrappers.scikit_learn import KerasRegressor

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from pandas.plotting import scatter_matrix
from statsmodels.graphics.tsaplots import plot_acf

In [40]:
import datetime as dt
import yfinance as yf

import pandas_datareader.data as web


In [41]:
stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']

start, end = dt.datetime(2010,1,1), dt.datetime(2024,1,28)
#stk_data = web.DataReader(stk_tickers, 'yahoo', start=start, end=end)
stk_data = yf.download(stk_tickers, start=start, end=end)
ccy_data = web.DataReader(ccy_tickers, 'fred', start=start, end=end)
idx_data = web.DataReader(idx_tickers, 'fred', start=start, end=end)

[*********************100%%**********************]  3 of 3 completed


TypeError: download() got multiple values for argument 'start'

In [43]:
stk_data

Price        Adj Close                               Close              \
Ticker           GOOGL         IBM        MSFT       GOOGL         IBM   
Date                                                                     
2010-01-04   15.684434   77.467285   23.474920   15.684434  126.625237   
2010-01-05   15.615365   76.531502   23.482504   15.615365  125.095604   
2010-01-06   15.221722   76.034348   23.338396   15.221722  124.282982   
2010-01-07   14.867367   75.771149   23.095678   14.867367  123.852776   
2010-01-08   15.065566   76.531502   23.254959   15.065566  125.095604   
...                ...         ...         ...         ...         ...   
2024-01-22  145.990005  172.830002  396.510010  145.990005  172.830002   
2024-01-23  147.039993  173.940002  398.899994  147.039993  173.940002   
2024-01-24  148.699997  173.929993  402.559998  148.699997  173.929993   
2024-01-25  151.869995  190.429993  404.869995  151.869995  190.429993   
2024-01-26  152.190002  187.419998  403.929993  152.190002  187.419998   

Price                         High                                 Low  \
Ticker            MSFT       GOOGL         IBM        MSFT       GOOGL   
Date                                                                     
2010-01-04   30.950001   15.753504  127.122368   31.100000   15.621622   
2010-01-05   30.959999   15.711712  126.051628   31.100000   15.554054   
2010-01-06   30.770000   15.662162  125.707458   31.080000   15.174174   
2010-01-07   30.450001   15.265265  124.521988   30.700001   14.831081   
2010-01-08   30.660000   15.096346  125.162521   30.879999   14.742492   
...                ...         ...         ...         ...         ...   
2024-01-22  396.510010  148.389999  174.449997  400.619995  145.839996   
2024-01-23  398.899994  147.179993  174.020004  399.380005  145.500000   
2024-01-24  402.559998  149.850006  174.860001  405.630005  148.100006   
2024-01-25  404.869995  153.050003  196.899994  407.010010  149.539993   
2024-01-26  403.929993  152.539993  192.389999  406.170013  151.009995   

Price                                     Open                          \
Ticker             IBM        MSFT       GOOGL         IBM        MSFT   
Date                                                                     
2010-01-04  125.095604   30.590000   15.689439  125.411087   30.620001   
2010-01-05  124.378586   30.639999   15.695195  125.889099   30.850000   
2010-01-06  124.101341   30.520000   15.662162  124.933075   30.879999   
2010-01-07  123.240921   30.190001   15.250250  124.158699   30.629999   
2010-01-08  123.374763   30.240000   14.814815  123.393883   30.280001   
...                ...         ...         ...         ...         ...   
2024-01-22  172.399994  393.589996  147.100006  172.820007  400.019989   
2024-01-23  172.479996  393.929993  145.889999  172.899994  395.750000   
2024-01-24  172.899994  400.450012  148.539993  174.759995  401.540009   
2024-01-25  184.830002  402.529999  150.070007  184.960007  404.320007   
2024-01-26  186.160004  402.429993  151.100006  191.309998  404.369995   

Price          Volume                      
Ticker          GOOGL       IBM      MSFT  
Date                                       
2010-01-04   78169752   6438444  38409100  
2010-01-05  120067812   7156104  49749600  
2010-01-06  158988852   5863144  58182400  
2010-01-07  256315428   6109268  50559700  
2010-01-08  188783028   4390271  51197400  
...               ...       ...       ...  
2024-01-22   32200400   4926000  27016900  
2024-01-23   21636100   3983500  20525900  
2024-01-24   25233500   9097800  24867000  
2024-01-25   29149100  29596200  21021200  
2024-01-26   26101000   9890500  17786700  

[3540 rows x 18 columns]